In [ ]:
from minio import Minio
from minio.error import S3Error

In [ ]:
def conectar_minio(endpoint='localhost:9000', 
                  access_key=None, 
                  secret_key=None, 
                  secure=False):
    """
    Establece una conexión con un servidor MinIO.
    
    Parámetros:
    -----------
    endpoint : str, opcional
        Dirección del servidor MinIO (por defecto 'localhost:9000')
    access_key : str, requerido
        Usuario/access key para la autenticación
    secret_key : str, requerido
        Contraseña/secret key para la autenticación
    secure : bool, opcional
        Si es True, usa HTTPS. Si es False, usa HTTP (por defecto False)
    
    Retorna:
    --------
    Minio
        Cliente MinIO conectado
    
    Excepciones:
    ------------
    ValueError
        Si no se proporcionan access_key o secret_key
    S3Error
        Si hay un error al conectar con el servidor MinIO
    """
    
    if not access_key or not secret_key:
        raise ValueError("Se requieren access_key y secret_key para la conexión")
    
    try:
        # Crear cliente MinIO
        cliente = Minio(
            endpoint,
            access_key=access_key,
            secret_key=secret_key,
            secure=secure
        )
        
        # Verificar conexión listando los buckets (opcional)
        buckets = cliente.list_buckets()
        print(f"Conexión exitosa a MinIO en {endpoint}")
        print(f"Buckets disponibles: {[bucket.name for bucket in buckets]}")
        
        return cliente
    
    except S3Error as err:
        print(f"Error al conectar con MinIO: {err}")
        raise

Leer archivo desde minio Minio

In [ ]:
import PyPDF2
import docx
from io import BytesIO
import pandas as pd

def extraer_archivo_minio(minio_client, bucket_name, ruta_archivo, tipo_archivo, encoding='utf-8'):
    """
    Extrae un archivo de MinIO y lo retorna en el formato adecuado.
    Parámetros:
    -----------
    minio_client : Minio
        Cliente MinIO ya conectado
    bucket_name : str
        Nombre del bucket donde está el archivo
    ruta_archivo : str
        Ruta completa del archivo dentro del bucket
    tipo_archivo : str
        Tipo de archivo a extraer ('pdf', 'word', 'excel', 'csv', 'parquet')
    encoding : str, opcional
        Codificación para archivos de texto (por defecto 'utf-8')
    Retorna:
    --------
    Depende del tipo de archivo:
    - 'pdf': Texto extraído (str)
    - 'word': Documento de python-docx
    - 'excel': DataFrame de pandas
    - 'csv': DataFrame de pandas
    - 'parquet': DataFrame de pandas
    Excepciones:
    ------------
    ValueError
        Si los parámetros son inválidos o el tipo no es soportado
    S3Error
        Si hay un error al acceder al archivo o el bucket no existe
    """
    if not minio_client:
        raise ValueError("Se requiere un cliente MinIO válido")
    if not bucket_name or not ruta_archivo or not tipo_archivo:
        raise ValueError("bucket_name, ruta_archivo y tipo_archivo son requeridos")
    tipo_archivo = tipo_archivo.lower()
    tipos_soportados = ['pdf', 'word', 'excel', 'csv', 'parquet']
    if tipo_archivo not in tipos_soportados:
        raise ValueError(f"Tipo de archivo '{tipo_archivo}' no soportado. Use: {tipos_soportados}")
    try:
        response = minio_client.get_object(bucket_name, ruta_archivo)
        data = BytesIO(response.read())
        data.seek(0)
        if tipo_archivo == 'pdf':
            pdf_reader = PyPDF2.PdfReader(data)
            text = "\n".join([page.extract_text() for page in pdf_reader.pages])
            return text
        elif tipo_archivo == 'word':
            return docx.Document(data)
        elif tipo_archivo == 'excel':
            return pd.read_excel(data)
        elif tipo_archivo == 'csv':
            try:
                return pd.read_csv(data, encoding=encoding)
            except UnicodeDecodeError:
                # Prueba con latin-1 si utf-8 falla
                data.seek(0)
                return pd.read_csv(data, encoding='latin-1')
        elif tipo_archivo == 'parquet':
            return pd.read_parquet(data)
    except S3Error as err:
        print(f"Error al acceder al archivo en MinIO: {err}")
        raise
    except Exception as e:
        print(f"Error al procesar el archivo {ruta_archivo}: {e}")
        raise
    finally:
        response.close()
        response.release_conn()

Generar ruta 

In [ ]:
from datetime import datetime

def generar_ruta_fecha(separador='/', fecha=None):
    """
    Genera un string con la fecha en formato año/mes/día.
    
    Parámetros:
    -----------
    separador : str, opcional
        Carácter separador entre componentes (por defecto '/')
    fecha : datetime, opcional
        Fecha específica a formatear (si None, usa fecha actual)
    
    Retorna:
    --------
    str
        String con el formato 'YYYY{separador}MM{separador}DD'
    """
    # Usar fecha actual si no se proporciona una específica
    fecha_a_usar = fecha if fecha is not None else datetime.now()
    
    # Formatear la fecha
    ruta_fecha = fecha_a_usar.strftime(f"%Y{separador}%m{separador}%d")
    
    return ruta_fecha

In [ ]:
ruta = generar_ruta_fecha()
print(ruta)  

Subir archivos Minio

In [ ]:
from io import BytesIO, StringIO
import pandas as pd
from minio.error import S3Error  # Asegúrate de importar S3Error

def guardar_df_en_minio(minio_client, df, bucket_name, ruta_destino, 
                        formato='parquet', crear_bucket=False):
    """
    Guarda un DataFrame directamente en un bucket de MinIO.
    """
    
    if not minio_client:
        raise ValueError("Se requiere un cliente MinIO válido")
    if not isinstance(df, pd.DataFrame):
        raise ValueError("El parámetro df debe ser un pandas.DataFrame")
    if not bucket_name or not ruta_destino:
        raise ValueError("bucket_name y ruta_destino son requeridos")
    
    formatos_soportados = {
        'parquet': {
            'mime': 'application/parquet',
            'writer': lambda buffer: df.to_parquet(buffer, index=False),
            'extension': '.parquet',
            'buffer_type': BytesIO
        },
        'csv': {
            'mime': 'text/csv',
            'writer': lambda buffer: df.to_csv(buffer, index=False),
            'extension': '.csv',
            'buffer_type': StringIO
        },
        'json': {
            'mime': 'application/json',
            'writer': lambda buffer: df.to_json(buffer, orient='records'),
            'extension': '.json',
            'buffer_type': BytesIO
        },
        'excel': {
            'mime': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
            'writer': lambda buffer: df.to_excel(buffer, index=False),
            'extension': '.xlsx',
            'buffer_type': BytesIO
        }
    }
    
    formato = formato.lower()
    if formato not in formatos_soportados:
        raise ValueError(f"Formato '{formato}' no soportado. Use: {list(formatos_soportados.keys())}")
    
    if not ruta_destino.lower().endswith(formatos_soportados[formato]['extension']):
        ruta_destino += formatos_soportados[formato]['extension']
    
    try:
        if crear_bucket and not minio_client.bucket_exists(bucket_name):
            minio_client.make_bucket(bucket_name)
            print(f"Bucket '{bucket_name}' creado exitosamente")

        if not minio_client.bucket_exists(bucket_name):
            raise S3Error(f"El bucket '{bucket_name}' no existe", bucket_name, None, 404)
        
        # Crear buffer en memoria
        buffer = formatos_soportados[formato]['buffer_type']()
        formatos_soportados[formato]['writer'](buffer)

        # Ajuste para CSV (convertir a BytesIO)
        if formato == 'csv':
            buffer.seek(0)
            data = BytesIO(buffer.getvalue().encode('utf-8'))
            length = len(data.getvalue())
        else:
            buffer.seek(0)
            data = buffer
            length = buffer.getbuffer().nbytes
        
        # Subir a MinIO
        minio_client.put_object(
            bucket_name=bucket_name,
            object_name=ruta_destino,
            data=data,
            length=length,
            content_type=formatos_soportados[formato]['mime']
        )
        
        ruta_completa = f"{bucket_name}/{ruta_destino}"
        print(f"DataFrame guardado exitosamente en: {ruta_completa}")
        return ruta_completa

    except S3Error as err:
        print(f"Error al guardar DataFrame en MinIO: {err}")
        raise
    except Exception as e:
        print(f"Error al procesar el DataFrame: {e}")
        raise
